In [43]:
pip install langchain openai langchain-community

Note: you may need to restart the kernel to use updated packages.


In [51]:
# install the required libraries
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import asyncio

# Summary
- This Python code utilizes Langchain and OpenAI's LLM to create an AI-powered system for processing tweets.
- It classifies tweets into categories (question, feedback, complaint, appreciation), analyzes their sentiment (positive, negative, neutral), and generates appropriate responses.
- The code employs asynchronous programming (async and await) to handle multiple tweets concurrently, improving efficiency. The nest_asyncio library ensures compatibility with interactive environments like Jupyter notebooks

# LLM Initialization
- This line initializes the Large Language Model (LLM) that will be used for all the subsequent tasks.
- OpenAI() creates an instance of the OpenAI LLM. You'll need to have the openai Python library installed (pip install openai) and have your OpenAI API key configured as an environment variable (OPENAI_API_KEY).
- temperature=0.7 is a parameter that controls the "creativity" of the LLM's output. A lower temperature (closer to 0) makes the output more deterministic and predictable, while a higher temperature (closer to 1) makes the output more creative and unpredictable.

In [59]:
# Initialize the LLM
llm = OpenAI(temperature=0.7)

# Tweet Classifier (TweetClassifier class):
- This class is responsible for classifying tweets into predefined categories.
- __init__(self, llm): The constructor takes the LLM instance as input and sets up the prompt template and the LLM chain.
  - PromptTemplate: Defines the template for the prompt that will be sent to the LLM. The {tweet} placeholder will be replaced with the actual tweet text.
  - LLMChain: Combines the LLM and the prompt template to create a chain that can be easily used to generate LLM responses.
- async def classify(self, tweet: str) -> str: This is an asynchronous function that takes a tweet as input and returns the predicted category.
  - await self.chain.arun(tweet=tweet): This line uses the LLM chain to send the tweet to the LLM for classification. arun is the asynchronous version of run. The await keyword pauses execution until the LLM responds.

# Sentiment Analyzer (SentimentAnalyzer class):
- This class is very similar to TweetClassifier, but it's responsible for analyzing the sentiment of a tweet (Positive, Negative, or Neutral). It uses a different prompt template tailored for sentiment analysis.

# Response Generator (ResponseGenerator class):
- This class generates a response to a tweet, taking into account the tweet itself, its category, and its sentiment. It also uses a different prompt template.

# Twitter Manager (TwitterManager class):
- This class acts as an orchestrator. It manages the entire tweet processing workflow.
- __init__(self, llm): The constructor takes the LLM and creates instances of the TweetClassifier, SentimentAnalyzer, and ResponseGenerator.
- async def handle_tweet(self, tweet: str): This asynchronous function takes a tweet as input and performs the following steps: 
  - Classifies the tweet using self.classifier.classify(tweet).
  - Analyzes the sentiment using self.sentiment_analyzer.analyze(tweet).
  - Generates a response using self.response_generator.generate_response(tweet, category, sentiment).
  - Returns a dictionary containing the original tweet, its category, sentiment, and the generated response.

# Testing the System (main function):
- This asynchronous function creates a TwitterManager instance and a list of sample tweets.
- It then iterates through the tweets, calling manager.handle_tweet(tweet) for each tweet (using await because handle_tweet is asynchronous).
- Finally, it prints the results for each tweet.

# Running the Main Function:
- if __name__ == "__main__":: This block ensures that the code inside it only runs when the script is executed directly (not when it's imported as a module).
- import asyncio: Imports the asyncio library for asynchronous programming.
- import nest_asyncio: Imports the nest_asyncio library. This is crucial for running asynchronous code in environments like Jupyter notebooks, which often have their own running event loops. nest_asyncio allows you to nest event loops.
- nest_asyncio.apply(): Applies the nest_asyncio patch, enabling nested event loops.
- asyncio.run(main()): Runs the main function using asyncio.run(), which creates and manages the event loop.

In [66]:
# 1. The Sorting Agent (Classification)
class TweetClassifier:
    def __init__(self, llm):
        self.prompt = PromptTemplate(
            input_variables=["tweet"],
            template="""
            Classify this tweet into one of these categories:
            - Question
            - Feedback
            - Complaint
            - Appreciation
            
            Tweet: {tweet}
            Category:"""
        )
        self.chain = LLMChain(llm=llm, prompt=self.prompt)
    
    async def classify(self, tweet: str) -> str:
        return await self.chain.arun(tweet=tweet)

# 2. The Mood Detective (Sentiment Analysis)
class SentimentAnalyzer:
    def __init__(self, llm):
        self.prompt = PromptTemplate(
            input_variables=["tweet"],
            template="""
            Analyze the sentiment of this tweet and return either POSITIVE, NEGATIVE, or NEUTRAL.
            
            Tweet: {tweet}
            Sentiment:"""
        )
        self.chain = LLMChain(llm=llm, prompt=self.prompt)
    
    async def analyze(self, tweet: str) -> str:
        return await self.chain.arun(tweet=tweet)

# 3. The Reply Writer (Response Generator)
class ResponseGenerator:
    def __init__(self, llm):
        self.prompt = PromptTemplate(
            input_variables=["tweet", "category", "sentiment"],
            template="""
            Create a friendly response to this tweet.
            Tweet: {tweet}
            Category: {category}
            Sentiment: {sentiment}
            
            Response:"""
        )
        self.chain = LLMChain(llm=llm, prompt=self.prompt)
    
    async def generate_response(self, tweet: str, category: str, sentiment: str) -> str:
        return await self.chain.arun(
            tweet=tweet,
            category=category,
            sentiment=sentiment
        )

# 4. The Boss Agent (Orchestrator)
class TwitterManager:
    def __init__(self, llm):
        self.classifier = TweetClassifier(llm)
        self.sentiment_analyzer = SentimentAnalyzer(llm)
        self.response_generator = ResponseGenerator(llm)
    
    async def handle_tweet(self, tweet: str):
        category = await self.classifier.classify(tweet)
        sentiment = await self.sentiment_analyzer.analyze(tweet)
        response = await self.response_generator.generate_response(tweet, category, sentiment)
        
        return {
            "tweet": tweet,
            "category": category,
            "sentiment": sentiment,
            "response": response
        }

# Testing the System
async def main():
    manager = TwitterManager(llm)
    tweets = [
        "Your product is amazing! Just what I needed 😊",
        "How do I reset my password? It's not working 😕",
        "This product sucks.... 😕",
        "Been waiting for support for 2 days now... 😠"
    ]
    
    for tweet in tweets:
        result = await manager.handle_tweet(tweet)
        print("\nTweet:", result["tweet"])
        print("Category:", result["category"])
        print("Sentiment:", result["sentiment"])
        print("Response:", result["response"])

# Run the main function
import asyncio

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()  # Allows nested event loops
    asyncio.run(main())





Tweet: Your product is amazing! Just what I needed 😊
Category:  Appreciation
Sentiment:  POSITIVE
Response:  We're so happy to hear that our product met your needs! Nothing makes us happier than a satisfied customer 🙂

Tweet: How do I reset my password? It's not working 😕
Category:  Question
Sentiment:  NEUTRAL
Response:  I'm sorry to hear that you're having trouble logging in. We can definitely help you reset your password. Have you tried using the "forgot password" option on the login page? Let me know if you still need assistance! 🙂

Tweet: This product sucks.... 😕
Category:  Complaint 
Sentiment:  NEGATIVE
Response:  I'm sorry to hear that you're not satisfied with our product! Could you tell me more about what went wrong? We value your feedback and would love to improve your experience with our product.

Tweet: Been waiting for support for 2 days now... 😠
Category:  Complaint
Sentiment:  NEGATIVE
Response: 
Hey there! We're sorry to hear that you've been waiting for support for 2